<a href="https://colab.research.google.com/github/lawalAfeez820/NLP/blob/main/word_level_text_gen%20using%20lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A WORD LEVEL TEXT GENERATION .

## Language model is the probability distribution over a sequence of words.There are wo form of generating text which are character level and word level text generation.Due to the time conputation of character level text generation, i decided to develop an NLP word level text generation.

## Text generation works by providing seed sentence and this sentence will be used to generate the next possible word , the generated word is the joined to the initial seed sentence to make for the model to generate another word that follows. Here is a description.

#### seed sentence: I will
#### with the seed sentence above(I will), next posssible word will be generated say "go". The "go" will be appended to the initial sentence to create a new seed sentence "I will go" . This new sentence will then be used to generated another next posible word. Generally, It is text generation is a reculsive process.

## Next Word can be generated using sampling(Picking next word base on the conditional probability distribution).There are various way of sampling, Here, I used greedy search and top_k sampling method.

## I used tensorflow data api during the implention for fast processing using(cache and prefecth), I also used keras text preprocessing layer for data cleaning and standadization(lowering the alphabet,removing html tags etc.).A simple LSTM  was used during the model building.

## Shortcomings: The model was trained with corpus containing few words due to the limitation of colab so it is possible to generate some unneccessary word.A more robost way of getting best result is by using lstm endoder decoder or transformer which will be implemented soon.

In [ ]:
import tensorflow as tf

!curl -O https://s3.amazonaws.com/text-datasets/nietzsche.txt


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  586k  100  586k    0     0  2197k      0 --:--:-- --:--:-- --:--:-- 2206k


In [ ]:
# loading the dataset line by line with tensorflow data api
text_data=tf.data.TextLineDataset(["nietzsche.txt"])

In [ ]:

for i in text_data.take(7):
  print(i.numpy().decode("utf-8"))

PREFACE


SUPPOSING that Truth is a woman--what then? Is there not ground
for suspecting that all philosophers, in so far as they have been
dogmatists, have failed to understand women--that the terrible
seriousness and clumsy importunity with which they have usually paid


In [ ]:
#converting each line to a list of strngs
raw_data=text_data.map(lambda x: tf.strings.split(x))


In [ ]:
for element in raw_data.take(5):
  print(element.numpy())

[b'PREFACE']
[]
[]
[b'SUPPOSING' b'that' b'Truth' b'is' b'a' b'woman--what' b'then?' b'Is'
 b'there' b'not' b'ground']
[b'for' b'suspecting' b'that' b'all' b'philosophers,' b'in' b'so' b'far'
 b'as' b'they' b'have' b'been']


In [ ]:
#flattening generatedted list
raw_data=raw_data.flat_map(lambda x: tf.data.Dataset.from_tensor_slices(x))
for i in raw_data.take(5):
  print(i.numpy()) 

b'PREFACE'
b'SUPPOSING'
b'that'
b'Truth'
b'is'


In [ ]:
# using the window size fuction on the dataset for the number of sequence to use.
#the input_size is the number dependent variable for predicting thenext word.
#notice that the first variable inside the window function is input_size+1, the one is due to the next word to be predicted


input_size=4

sequence_data=raw_data.window(input_size+1,drop_remainder=True)

for window in sequence_data.take(3):
  print(list(window.as_numpy_iterator()))


[b'PREFACE', b'SUPPOSING', b'that', b'Truth', b'is']
[b'a', b'woman--what', b'then?', b'Is', b'there']
[b'not', b'ground', b'for', b'suspecting', b'that']


In [ ]:


sequence_data=sequence_data.flat_map(lambda x: x.batch(5))

In [ ]:
for i in sequence_data.take(5):
  print(i)

tf.Tensor([b'PREFACE' b'SUPPOSING' b'that' b'Truth' b'is'], shape=(5,), dtype=string)
tf.Tensor([b'a' b'woman--what' b'then?' b'Is' b'there'], shape=(5,), dtype=string)
tf.Tensor([b'not' b'ground' b'for' b'suspecting' b'that'], shape=(5,), dtype=string)
tf.Tensor([b'all' b'philosophers,' b'in' b'so' b'far'], shape=(5,), dtype=string)
tf.Tensor([b'as' b'they' b'have' b'been' b'dogmatists,'], shape=(5,), dtype=string)


In [ ]:
#creating x nd y variable for model fitting preparation
sequence_data=sequence_data.map(lambda x : (x[:-1], x[-1]))

x_data=sequence_data.map(lambda x,y:x)

y_data=sequence_data.map(lambda x,y:y)

In [ ]:
for i in x_data.take(3):
  print(i)

tf.Tensor([b'PREFACE' b'SUPPOSING' b'that' b'Truth'], shape=(4,), dtype=string)
tf.Tensor([b'a' b'woman--what' b'then?' b'Is'], shape=(4,), dtype=string)
tf.Tensor([b'not' b'ground' b'for' b'suspecting'], shape=(4,), dtype=string)


In [ ]:
# reshaping the x dataset

def convert_string(x: tf.Tensor):
  str1=""
  for element in x:
    str1+=element.numpy().decode("utf-8")+" "
  str1=tf.convert_to_tensor(str1[:-1])
  return str1

In [ ]:
x_data=x_data.map(lambda x: tf.py_function(func=convert_string,inp=[x],Tout=tf.string))

In [ ]:
for element in x_data.take(5):
  print(element)

tf.Tensor(b'PREFACE SUPPOSING that Truth', shape=(), dtype=string)
tf.Tensor(b'a woman--what then? Is', shape=(), dtype=string)
tf.Tensor(b'not ground for suspecting', shape=(), dtype=string)
tf.Tensor(b'all philosophers, in so', shape=(), dtype=string)
tf.Tensor(b'as they have been', shape=(), dtype=string)


In [ ]:
print(x_data.element_spec,y_data.element_spec)

TensorSpec(shape=<unknown>, dtype=tf.string, name=None) TensorSpec(shape=(), dtype=tf.string, name=None)


In [ ]:
for i in x_data.map(lambda x : tf.reshape(x,[1])).take(2):
  print(i)

tf.Tensor([b'PREFACE SUPPOSING that Truth'], shape=(1,), dtype=string)
tf.Tensor([b'a woman--what then? Is'], shape=(1,), dtype=string)


In [ ]:
x_data=x_data.map(lambda x : tf.reshape(x,[1]))

In [ ]:
# preprocessing the dataset with keras preprocessing layer

#Preprocessing the text using preprocessing layer text vectorization

import re,string
def custom_standadization(text):

  lowercase=tf.strings.lower(text)

  stripped_html=tf.strings.regex_replace(lowercase,"<br />" ," ")

  stripped_number=tf.strings.regex_replace(stripped_html,"[\d-]"," ")

  stripped_punc=tf.strings.regex_replace(stripped_number,"[%s]" % re.escape(string.punctuation),"")

  return stripped_punc

In [ ]:
from keras.layers import TextVectorization

vectorizer=TextVectorization(standardize=custom_standadization,
                             output_sequence_length=4
)

In [ ]:
# training the vectorizer

vectorizer.adapt(raw_data.batch(batch_size=128))

In [ ]:
# lenth of the vocabulary
len(vectorizer.get_vocabulary())

9903

In [ ]:
# function that will be aplied to each element in the tensor

def vectorize_text(text):
  text=tf.expand_dims(text,1)
  return tf.squeeze(vectorizer(text))

In [ ]:
#checking some examples of the x_data
for i in x_data.take(5):
  print(i)

tf.Tensor([b'PREFACE SUPPOSING that Truth'], shape=(1,), dtype=string)
tf.Tensor([b'a woman--what then? Is'], shape=(1,), dtype=string)
tf.Tensor([b'not ground for suspecting'], shape=(1,), dtype=string)
tf.Tensor([b'all philosophers, in so'], shape=(1,), dtype=string)
tf.Tensor([b'as they have been'], shape=(1,), dtype=string)


In [ ]:
# applying the vectorizer to the train data
x_data=x_data.map(vectorize_text)

y_data=y_data.map(vectorizer)



In [ ]:

# checking the examples

for i in x_data.take(2):
  print(i)

tf.Tensor([4041  576    9  119], shape=(4,), dtype=int64)
tf.Tensor([  8 147  41 143], shape=(4,), dtype=int64)


In [ ]:
for i in y_data.take(2):
  print(i)


tf.Tensor([7 0 0 0], shape=(4,), dtype=int64)
tf.Tensor([40  0  0  0], shape=(4,), dtype=int64)


In [ ]:
# only one value is required for the y_data, the transformation below select only the first value of y_data
y_data=y_data.map(lambda x: x[:1])


In [ ]:
for i in y_data.take(2):
  print(i)

tf.Tensor([7], shape=(1,), dtype=int64)
tf.Tensor([40], shape=(1,), dtype=int64)


In [ ]:
# joining both x_data and y_data together for optimization and shape 
train_ds=tf.data.Dataset.zip((x_data,y_data))
print(train_ds.element_spec)

(TensorSpec(shape=<unknown>, dtype=tf.int64, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))


In [ ]:
# function to fix the shape after zipping
def fixed_shape(x,y):
  x.set_shape([4])
  y.set_shape([1])
  return x,y

In [ ]:
# applying the fix shape function
train_ds=train_ds.map(fixed_shape)
print(train_ds.element_spec)

(TensorSpec(shape=(4,), dtype=tf.int64, name=None), TensorSpec(shape=(1,), dtype=tf.int64, name=None))


In [ ]:
train_ds.cardinality

<bound method DatasetV2.cardinality of <MapDataset element_spec=(TensorSpec(shape=(4,), dtype=tf.int64, name=None), TensorSpec(shape=(1,), dtype=tf.int64, name=None))>>

In [ ]:
#tensorflow data pipeline for optimizing the process

from tensorflow.python.data.ops.dataset_ops import AUTOTUNE
autotune=AUTOTUNE
train_ds=train_ds.shuffle(512).padded_batch(128,drop_remainder=True).cache().prefetch(autotune)

In [ ]:
tf.data.Dataset.padded_batch

<function tensorflow.python.data.ops.dataset_ops.DatasetV2.padded_batch>

In [ ]:
train_ds.element_spec

(TensorSpec(shape=(128, 4), dtype=tf.int64, name=None),
 TensorSpec(shape=(128, 1), dtype=tf.int64, name=None))

# Model Building

In [ ]:
import numpy as np
from keras.models import Model
from keras.layers import LSTM,Input,Embedding,Flatten,Dense,Lambda


input=Input(shape=4)
embedding=Embedding(len(vectorizer.get_vocabulary()),output_dim=40)(input)

encoder=LSTM(units=64,return_sequences=True)(embedding)
flatten=Flatten()(encoder)
output=Dense(len(vectorizer.get_vocabulary()),activation="softmax")(flatten)

model=Model(input,output)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 4)]               0         
                                                                 
 embedding (Embedding)       (None, 4, 40)             396120    
                                                                 
 lstm (LSTM)                 (None, 4, 64)             26880     
                                                                 
 flatten (Flatten)           (None, 256)               0         
                                                                 
 dense (Dense)               (None, 9903)              2545071   
                                                                 
Total params: 2,968,071
Trainable params: 2,968,071
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer="adam",loss="sparse_categorical_crossentropy",metrics=["accuracy"])

In [ ]:
model.fit(train_ds,epochs=63,batch_size=128)

Epoch 1/63
154/154 [==============================] - 45s 272ms/step - loss: 7.6089 - accuracy: 0.0596
Epoch 2/63
154/154 [==============================] - 13s 86ms/step - loss: 6.4888 - accuracy: 0.0616
Epoch 3/63
154/154 [==============================] - 13s 87ms/step - loss: 6.4024 - accuracy: 0.0616
Epoch 4/63
154/154 [==============================] - 14s 88ms/step - loss: 6.3429 - accuracy: 0.0616
Epoch 5/63
154/154 [==============================] - 25s 159ms/step - loss: 6.2947 - accuracy: 0.0614
Epoch 6/63
154/154 [==============================] - 18s 117ms/step - loss: 6.2517 - accuracy: 0.0619
Epoch 7/63
154/154 [==============================] - 18s 116ms/step - loss: 6.2063 - accuracy: 0.0675
Epoch 8/63
154/154 [==============================] - 20s 127ms/step - loss: 6.1479 - accuracy: 0.0795
Epoch 9/63
154/154 [==============================] - 20s 130ms/step - loss: 6.0642 - accuracy: 0.0840
Epoch 10/63
154/154 [==============================] - 23s 148ms/step - loss

In [ ]:
def decode_sequence(encode_sequence):
  decoded_sequence=[]

  for token in encode_sequence:
    decoded_sequence.append(vectorizer.get_vocabulary()[token])
  sequence=" ".join(decoded_sequence)
  return sequence

In [ ]:
decode_sequence([2,3,4])

'the of and'

In [ ]:
text="We are in "
a=vectorizer(text).numpy()
a=np.array(a).reshape(1,4)
print(a)

[[29 20  6  0]]


In [ ]:
vectorizer.get_vocabulary()[np.argmax(model.predict(a))]

'a'

In [ ]:
 # SAMPLING METHOD FOR THE TEXT GENERATION

def softmax(z):
  return np.exp(z)/sum(np.exp(z))

def greedy_search(conditional_probability):

  return np.argmax(conditional_probability)



def top_k_sampling(conditional_probability,k):

  top_k_sampling,index=tf.math.top_k(conditional_probability,k=k)

  top_k_sampling=np.asarray(top_k_sampling).astype("float32")

  top_k_sampling=np.squeeze(top_k_sampling)

  index=np.asarray(index).astype("int32")

  redistributed_k=softmax(top_k_sampling)

  redistributed_k=np.asarray(redistributed_k).astype("float32")

  sampled_token=np.random.choice(np.squeeze(index),p=redistributed_k)

  return sampled_token

In [ ]:
vectorizer.get_vocabulary()[top_k_sampling(model.predict(a),3)]

'good'

In [ ]:

def generate_text(model,seed,text_size):

  count_purpose=seed.split()
  list_of_text=seed.split()


  for i in range(text_size):

    if len(list_of_text)>=4:
      

      vector=vectorizer(" ".join(list_of_text[-4:]))

      vector=np.array(vector).reshape(1,4)

      conditional_probability=model.predict(vector)

      output=vectorizer.get_vocabulary()[top_k_sampling(conditional_probability,3)]

      list_of_text.append(output)

    else:

      vector=vectorizer(" ".join(list_of_text))

      vector=np.array(vector).reshape(1,4)

      conditional_probability=model.predict(vector)

      output=vectorizer.get_vocabulary()[top_k_sampling(conditional_probability,5)]

      list_of_text.append(output)

  return " ".join(list_of_text)

  







#TRYING OUT SOME SAMPLES

In [ ]:
# generated text using top_k sampling(decided not to use greedy search)
generate_text(model,"we need to try our luck today",5)

'we need to try our luck today so it be without been'

In [ ]:
generate_text(model,"let's try something out",3)

"let's try something out in a mud"

In [ ]:
generate_text(model,"let's try something out",3)

"let's try something out for that so"

In [ ]:
generate_text(model,"i will try and check him when i am back",3)

'i will try and check him when i am back to his sympathy'

In [ ]:
generate_text(model,"i will try and check him when i am back",15)

'i will try and check him when i am back to will to be oneself of the queen not have be been says for fastidious'